In [1]:
import pandas as pd
from tabula import read_pdf

# Extract properties for Agder lakes from PDF

Martin Hagen Ring has sent data describing the amount of lime (in tonnes) added to various lakes in Agder since the 1960s. He has also provided a PDF containing properties (volume, residence time etc.) for some of these lakes. See e-mail received 22.01.2024 for details.

This notebook extracts the lake properties from the PDF and saves them to Excel, along with the dataet describing lime amounts added.

In [2]:
# Read PDF data table
lake_df = read_pdf("../data/Dybdekart_samlet.pdf", pages=1)[0]
lake_df = lake_df.dropna(axis=1, how="all")
lake_df.columns = [
    "lake",
    "kommune",
    "nve_nr",
    "digitalised",
    "map_sheet:",
    "utm",
    "depth_map",
    "echo_sounder",
    "catch_area_m2",
    "ann_runoff_m",
    "ann_runoff_m3",
    "mean_q_m3ps",
    "lake_area_m2",
    "max_depth_m",
    "mean_depth_m",
    "volume_m3",
    "tau_years",
    "comment",
]

# Deal with strange values saved to PDF from Excel
lake_df = lake_df.query("tau_years != '#DIV/0!'")
lake_df = lake_df.applymap(
    lambda x: str(x).replace(",", ".") if isinstance(x, str) else x
)
lake_df = lake_df.applymap(
    lambda x: str(x).replace(" ", "") if isinstance(x, str) else x
)
for col in lake_df.columns:
    try:
        lake_df[col] = lake_df[col].astype(float)
    except ValueError:
        pass

# Just lakes with valid NVE ID and tau > 0
lake_df.dropna(subset=["nve_nr", "tau_years"], inplace=True)
lake_df = lake_df.query("tau_years > 0").reset_index(drop=True)

# Tidy
cols = [
    "lake",
    "kommune",
    "nve_nr",
    "utm",
    "catch_area_m2",
    "ann_runoff_m",
    "ann_runoff_m3",
    "mean_q_m3ps",
    "lake_area_m2",
    "max_depth_m",
    "mean_depth_m",
    "volume_m3",
    "tau_years",
]
lake_df = lake_df[cols]
lake_df

,lake,kommune,nve_nr,utm,catch_area_m2,ann_runoff_m,ann_runoff_m3,mean_q_m3ps,lake_area_m2,max_depth_m,mean_depth_m,volume_m3,tau_years
0,BEGERVANN,BIRKENES,10772.0,616792.0,4100000.0,1.00,4100000.0,0.13,760000.0,62.0,14.60,11096000.0,2.71
1,BELLANDSTJERN,BIRKENES,10539.0,472860.0,2000000.0,1.00,2000000.0,0.06,185000.0,14.5,2.70,499500.0,0.25
2,BJORVAtN,EVJE/HORNNES,10217.0,348940.0,5650000.0,1.20,6780000.0,0.21,420000.0,19.0,6.26,2629200.0,0.39
3,FLEKEVATN,FROLAND,10164.0,601951.0,1400000.0,1.00,1400000.0,0.04,531000.0,12.0,3.50,1858500.0,1.33
4,FØREVANN,FROLAND,1335.0,483982.0,10700000.0,1.05,11235000.0,0.36,1404000.0,24.0,5.95,8353800.0,0.74
5,GANGVATN,GRIMST/AREND,10636.0,712827.0,2250000.0,1.00,2250000.0,0.07,455000.0,25.0,6.55,2980250.0,1.32
6,GJEVDEN,ÅMLI,7863.0,526360.0,13500000.0,1.00,13500000.0,0.43,660000.0,21.0,6.70,4422000.0,0.33
7,HAUKOMVATN,BIRKENES,1334.0,NaN,18500000.0,1.20,22200000.0,0.70,1480000.0,60.0,16.90,25012000.0,1.13
8,HÅVATN,FROLAND,10313.0,722906.0,3000000.0,1.00,3000000.0,0.10,725000.0,61.0,18.40,13340000.0,4.45
9,HUNSDALSVANN,GRIMSTAD,10711.0,698808.0,2500000.0,1.00,2500000.0,0.08,360000.0,17.0,8.00,2880000.0,1.15


In [3]:
# Read liming data
lime_df = pd.read_excel("../data/agder_lime_added_raw.xlsx", sheet_name="data")
lime_df.head()

,lnr,forening,name,kommune,year,tonnes,method,lime,nve_nr,utm33_north,utm33_east,lat,lon
0,905,Hægebostad og Eiken JFF,Gletnevatnet,Hægebostad,1994,0.0,Båt,Kalksteinmel,1234,6502418,49049,58.4286,7.26775
1,1690,Vegårshei JFF,Kallbergsvatnet,Vegårdshei,1985,0.0,Båt,Kalksteinmel,8980,6532208,148917,58.7863,8.92145
2,1691,Vegårshei JFF,Kallbergsvatnet,Vegårdshei,1986,0.0,Båt,Kalksteinmel,8980,6532208,148917,58.7863,8.92145
3,1692,Vegårshei JFF,Kallbergsvatnet,Vegårdshei,1987,0.0,Båt,Kalksteinmel,8980,6532208,148917,58.7863,8.92145
4,1693,Vegårshei JFF,Kallbergsvatnet,Vegårdshei,1988,0.0,Båt,Kalksteinmel,8980,6532208,148917,58.7863,8.92145


In [4]:
# Filter lime data and lake data to only include lakes that match
nve_ids = list(
    set(lake_df["nve_nr"].astype(int).tolist()).intersection(
        set(lime_df["nve_nr"].astype(int).tolist())
    )
)
lake_df = lake_df.query("nve_nr in @nve_ids").reset_index(drop=True)
lime_df = lime_df.query("nve_nr in @nve_ids").reset_index(drop=True)

# Save
with pd.ExcelWriter('../data/agder_liming_data_tidy.xlsx') as writer:
    lake_df.to_excel(writer, sheet_name='lake_props', index=False)
    lime_df.to_excel(writer, sheet_name='lime_added', index=False)